In [1]:
install.packages("gtsummary")

also installing the dependencies ‘commonmark’, ‘xfun’, ‘V8’, ‘litedown’, ‘reactR’, ‘bigD’, ‘juicyjuice’, ‘markdown’, ‘reactable’, ‘cards’, ‘gt’


Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done



In [2]:
install.packages("gt")


Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done



In [22]:
install.packages("skimr")

Updating HTML index of packages in '.Library'

Making 'packages.html' ...
 done



In [6]:
library(haven)
library(stargazer)
library(dplyr)
library(infer)
library(ggplot2)
library(broom)
library(knitr)
library(kableExtra)
library(readr)
library(gtsummary)
library(gt)

ERROR: Error in library(kableExtra): there is no package called ‘kableExtra’


In [14]:
library(gt)
library(gtsummary)

In [7]:
data <- read_dta("./cen_ind_2021_pumf_v2.dta")

In [8]:
#removes NA rows
data <- data[data$CFInc != 88, ]

data <- data[data$agegrp != 88, ]

data <- data[data$genstat != 88, ]

data <- data[data$hdgree != 88, ]
data <- data[data$hdgree != 99, ]

data <- data[data$lfact != 88, ]
data <- data[data$lfact != 99, ]

data <- data[data$NOC21 != 88, ]
data <- data[data$NOC21 != 99, ]

data <- na.omit(data)


In [9]:
data <- data %>%
  mutate(first_gen = ifelse(genstat == 1, 1, 0))

data <- data %>%
  mutate(sec_gen = ifelse(genstat %in% c("2", "3"), 1, 0))
 
data <- data %>%
  mutate(third_gen = ifelse(genstat >= 4, 1, 0))


In [ ]:
colnames(data)


In [10]:
#making dummy for indivuals age 12-20
data <- data %>%
  mutate(agegrp_dummy = ifelse(agegrp %in% c("5", "6", "7"), 1, 0))

#dummy for bachelors degree to simplify 
data <- data %>%
  mutate(bachelors_degree = ifelse(hdgree %in% c(9), 1, 0))

In [ ]:
summary(data[, c("CFInc", "bachelors_degree", "genstat", "first_gen", 
                 "sec_gen", "third_gen", "Gender", "agegrp", "hdgree", 
                 "NOC21", "pr", "lfact")])


In [18]:
library(haven)

In [19]:

data_clean <- data %>%
  mutate(across(c(CFInc, hdgree, genstat, agegrp, Gender, NOC21, lfact), haven::as_factor))


In [20]:

my_table <- data_clean %>%
  select(CFInc, hdgree, genstat, agegrp, Gender, NOC21, lfact) %>%
  tbl_summary()

# Convert to gt object
gt_table <- as_gt(my_table)

# Save to HTML
gtsave(gt_table, "table_output.html")


In [24]:
library(skimr)

data_clean %>%
  select(agegrp, Gender, CFInc, hdgree) %>%
  skim()


── Data Summary ────────────────────────
                           Values    
Name                       Piped data
Number of rows             537314    
Number of columns          4         
_______________________              
Column type frequency:               
  factor                   4         
________________________             
Group variables            None      

── Variable type: factor ───────────────────────────────────────────────────────
  skim_variable n_missing complete_rate ordered n_unique
1 agegrp                0             1 FALSE         16
2 Gender                0             1 FALSE          2
3 CFInc                 0             1 FALSE         33
4 hdgree                0             1 FALSE         13
  top_counts                                      
1 35 : 58134, 30 : 58133, 25 : 55496, 40 : 55375  
2 Man: 279012, Wom: 258302                        
3 $15: 45892, $25: 40964, $20: 39095, $17: 33293  
4 Hig: 136834, Bac: 112178, Pro: 53692, No : 5

ERROR: Error in is.null(text_repr) || nchar(text_repr) == 0L: 'length = 7' in coercion to 'logical(1)'


In [ ]:
# simple regression 1 - CFInc (Income) based on Gentstat (First Gen / Second Gen Canadian)


reg_1 <-  lm(CFInc ~ genstat, data = data)

stargazer(reg_1, type = "text")


In [ ]:
# simple regression 2 - adding covariates for Gender and agegrp 

reg_2 <-  lm(CFInc ~ genstat + Gender + agegrp, data = data)

stargazer(reg_2, type = "text")


In [ ]:
# simple regression 3 - adding covariates for hdgree (education level) and lfact (labour force participation)

reg_3 <-  lm(CFInc ~ genstat + Gender + agegrp + hdgree + lfact, data = data)

stargazer(reg_3, type = "text")


In [ ]:
# simple regression 4 - by pr (province)

reg_4 <-  lm(CFInc ~ genstat + Gender + agegrp + hdgree + lfact + pr, data = data)

stargazer(reg_4, type = "text")


In [ ]:
reg_5 <-  lm(CFInc ~ genstat + Gender + agegrp + hdgree + lfact + pr, data = data)

stargazer(reg_4, type = "text")

In [ ]:
# checking for interaction effects 

reg_interaction <- lm(CFInc ~ genstat * hdgree + pr + Gender + agegrp + genstat * lfact, data = data)


stargazer(reg_interaction, type = "text")


In [ ]:
# more expierementing 

reg_interaction <- lm(CFInc ~ genstat * hdgree + pr + Gender + agegrp + genstat * lfact + hdgree * agegrp + hdgree * genstat, data = data)


stargazer(reg_interaction, type = "text")

In [ ]:
# more expierementing 

reg_interaction <- lm(CFInc ~ genstat * hdgree + pr + Gender + agegrp + hdgree * genstat, data = data)


stargazer(reg_interaction, type = "text")

In [ ]:
# Basic regressions
model1 <- lm(CFInc ~ genstat, data = data)
model2 <- lm(CFInc ~ genstat + Gender, data = data)
model3 <- lm(CFInc ~ genstat + Gender + agegrp, data = data)
model4 <- lm(CFInc ~ genstat + Gender + agegrp + hdgree, data = data)
model5 <- lm(CFInc ~ genstat * hdgree + pr + Gender + agegrp + genstat * lfact, data = data)

# Stargazer Table Output
stargazer(model1, model2, model3, model4, model5,
          type = "html", 
          title = "Income on Generational Status",
          dep.var.labels = "Dependent Variable: CFInc",
          out = "income_table5.html")



In [ ]:
# Basic regressions 2
model1 <- lm(CFInc ~ genstat, data = data)
model2 <- lm(CFInc ~ genstat + Gender + agegrp, data = data)
model3 <- lm(CFInc ~ genstat + Gender + agegrp + hdgree, data = data)
model4 <- lm(CFInc ~ genstat + Gender + agegrp + hdgree + NOC21, data = data)
model5 <- lm(CFInc ~ genstat + Gender + agegrp + hdgree + NOC21 + pr + genstat * lfact, data = data)

# Stargazer Table Output
stargazer(model1, model2, model3, model4,model5,
          type = "html", 
          title = "Income on Generational Status",
          dep.var.labels = "Dependent Variable: CFInc",
          out = "income_table12.html")



In [ ]:
#degree status on genstat 
# should be careful, meaning of hdgree is problematic 
model1.1 <- lm(bachelors_degree ~ genstat, data = data)
model2.1 <- lm(bachelors_degree ~ genstat + Gender, data = data)
model3.1 <- lm(bachelors_degree ~ genstat + Gender + agegrp, data = data)
model4.1 <- lm(bachelors_degree ~ genstat + Gender + agegrp + lfact*agegrp_dummy, data = data)

# Stargazer Table Output
stargazer(model1.1, model2.1, model3.1, model4.1, 
          type = "html", 
          title = "Degree on Generational Status",
          dep.var.labels = "Dependent Variable: bachelors_degree",
          out = "degree_table4.html")



In [ ]:
#very rough test if holding a bachelors degree is influenced by working ages 11-19
extra1=lm(bachelors_degree ~ lfact*agegrp_dummy, data = data)
summary(extra1)


In [ ]:
# sees income effect for first gen immegratns only 
# Basic regressions 2
model1 <- lm(CFInc ~ first_gen, data = data)
model2 <- lm(CFInc ~ first_gen + Gender + agegrp, data = data)
model3 <- lm(CFInc ~ first_gen + Gender + agegrp + hdgree, data = data)
model4 <- lm(CFInc ~ first_gen + Gender + agegrp + hdgree + NOC21, data = data)
model5 <- lm(CFInc ~ first_gen + Gender + agegrp + hdgree + NOC21 + pr + genstat * lfact, data = data)

# Stargazer Table Output
stargazer(model1, model2, model3, model4,model5,
        type = "text", 
          title = "Income on first Generational Status",
          dep.var.labels = "Dependent Variable: CFInc")


In [ ]:
# income for second gen immegrants only
model1 <- lm(CFInc ~ sec_gen, data = data)
model2 <- lm(CFInc ~ sec_gen + Gender + agegrp, data = data)
model3 <- lm(CFInc ~ sec_gen + Gender + agegrp + hdgree, data = data)
model4 <- lm(CFInc ~ sec_gen + Gender + agegrp + hdgree + NOC21, data = data)
model5 <- lm(CFInc ~ sec_gen + Gender + agegrp + hdgree + NOC21 + pr + genstat * lfact, data = data)

# Stargazer Table Output
stargazer(model1, model2, model3, model4,model5,
        type = "text", 
          title = "Income on Second Generational Status",
          dep.var.labels = "Dependent Variable: CFInc")

In [ ]:
# income for third gen immegrants only
model1 <- lm(CFInc ~ third_gen, data = data)
model2 <- lm(CFInc ~ third_gen + Gender + agegrp, data = data)
model3 <- lm(CFInc ~ third_gen + Gender + agegrp + hdgree, data = data)
model4 <- lm(CFInc ~ third_gen + Gender + agegrp + hdgree + NOC21, data = data)
model5 <- lm(CFInc ~ third_gen + Gender + agegrp + hdgree + NOC21 + pr + genstat * lfact, data = data)

# Stargazer Table Output
stargazer(model1, model2, model3, model4,model5,
        type = "text", 
          title = "Income on Third+ Generational Status",
          dep.var.labels = "Dependent Variable: CFInc")

In [ ]:
mod_A <- lm(CFInc ~ first_gen + Gender + agegrp + hdgree, data = data)
mod_B <- lm(CFInc ~ first_gen + Gender + agegrp + hdgree, data = data)
mod_C <- lm(CFInc ~ first_gen + Gender + agegrp + hdgree, data = data)


In [ ]:
#dummy for in labour force
data <- data %>%
  mutate(lfact_dummy = ifelse(lfact %in% c(11,12,13,14), 1, 0))



In [ ]:
#degree status on genstat 
# should be careful, meaning of hdgree is problematic 
model1 <- lm(bachelors_degree ~ third_gen, data = data)
model2 <- lm(bachelors_degree ~ third_gen + Gender, data = data)
model3 <- lm(bachelors_degree ~ third_gen + Gender + agegrp, data = data)
model4 <- lm(bachelors_degree ~ third_gen + Gender + agegrp + employed_dummy*agegrp_dummy, data = data)

# Stargazer Table Output
stargazer(model1, model2, model3, model4, 
          type = "text", 
          title = "Degree on Third Generational Status",
          dep.var.labels = "Dependent Variable: bachelors_degree",
          out = "degree_table4.html")


In [ ]:
#degree status on genstat 
# should be careful, meaning of hdgree is problematic 
model1 <- lm(bachelors_degree ~ first_gen, data = data)
model2 <- lm(bachelors_degree ~ first_gen + Gender, data = data)
model3 <- lm(bachelors_degree ~ first_gen + Gender + agegrp, data = data)
model4 <- lm(bachelors_degree ~ first_gen + Gender + agegrp + lfact*agegrp_dummy, data = data)

# Stargazer Table Output
stargazer(model1, model2, model3, model4, 
          type = "text", 
          title = "Degree on First Generational Status",
          dep.var.labels = "Dependent Variable: bachelors_degree",
          out = "degree_table4.html")


In [ ]:
#Plots of income distrabution for first seocnd and third gen immegrants overlapping 
fitted_A <- predict(mod_A)
fitted_B <- predict(mod_B)
fitted_C <- predict(mod_C)

# Get residual SD
sigma_A <- summary(mod_A)$sigma
sigma_B <- summary(mod_B)$sigma
sigma_C <- summary(mod_C)$sigma

# Simulate predicted outcomes around the fitted values
set.seed(123)
sim_A <- rnorm(length(fitted_A), mean = fitted_A, sd = sigma_A)
sim_B <- rnorm(length(fitted_B), mean = fitted_B, sd = sigma_B)
sim_C <- rnorm(length(fitted_C), mean = fitted_C, sd = sigma_C)

# Combine for plotting
df_plot <- data.frame(
  outcome = c(sim_A, sim_B, sim_C),
  group = factor(c(
    rep("A", length(sim_A)),
    rep("B", length(sim_B)),
    rep("C", length(sim_C))
  ))
)

# Plot
ggplot(df_plot, aes(x = outcome, fill = group)) +
  geom_density(alpha = 0.5) +
  labs(title = "Predicted Income Distrabution by Generational Immigrant Status",
       x = "Simulated Outcome", y = "Density") +
  theme_minimal()


In [ ]:
#degree status on genstat 
# should be careful, meaning of hdgree is problematic 
model1 <- lm(bachelors_degree ~ sec_gen, data = data)
model2 <- lm(bachelors_degree ~ sec_gen + Gender, data = data)
model3 <- lm(bachelors_degree ~ sec_gen + Gender + agegrp, data = data)
model4 <- lm(bachelors_degree ~ sec_gen + Gender + agegrp + lfact*agegrp_dummy, data = data)

# Stargazer Table Output
stargazer(model1, model2, model3, model4, 
          type = "text", 
          title = "Degree on First Second Generational Status",
          dep.var.labels = "Dependent Variable: bachelors_degree",
          out = "degree_table4.html")


In [ ]:
#degree status on genstat 
# should be careful, meaning of hdgree is problematic 
model1 <- lm(bachelors_degree ~ genstat, data = data)
model2 <- lm(bachelors_degree ~ genstat + Gender, data = data)
model3 <- lm(bachelors_degree ~ genstat + Gender + agegrp, data = data)
model4 <- lm(bachelors_degree ~ genstat + Gender + agegrp + lfact_dummy*agegrp_dummy, data = data)

# Stargazer Table Output
stargazer(model1, model2, model3, model4, 
          type = "html", 
          title = "Bachelors Degree on Generational Status",
          dep.var.labels = "Dependent Variable: bachelors_degree",
          out = "degree_table4.html")
